In [1]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import mlflow

import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

2023-04-25 11:30:40.370737: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 11:30:58.378636: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-25 11:31:23.910563: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-25 11:31:23.910762: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

In [2]:
import torch
print(torch.cuda.is_available())
print(tf.config.list_physical_devices("GPU"))

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-25 11:32:40.880214: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-25 11:32:42.851051: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-25 11:32:42.851621: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [16]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv("cleaned_data.csv")
LABEL = 'label'

# df > X, y
X, y = df.drop(LABEL, axis=1), df.loc[:, LABEL]

# encoding label
label_encoder = LabelEncoder()
label_encoder.fit(y)
df['label'] = y = label_encoder.transform(y) # range(7) > ['anger', 'disgust', 'fear', 'happiness', 'neutralism', 'sadness', 'surprise']

n_classes = len(label_encoder.classes_)

# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True, stratify=y)

trainset, testset = train_test_split(df, test_size=0.3, random_state=42, shuffle=True, stratify=df['label'])

In [14]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# https://huggingface.co/klue/bert-base

CHECKPOINT = "klue/bert-base"
bert_base = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, num_labels=n_classes)
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [17]:
import datasets
from datasets import Dataset

trainset = datasets.Dataset.from_pandas(trainset, preserve_index=False)
testset = datasets.Dataset.from_pandas(testset, preserve_index=False)

In [18]:
train_dataset = trainset.map(lambda x : tokenizer(x['text'], padding='max_length', truncation=True, max_length=256))
test_dataset = testset.map(lambda x : tokenizer(x['text'], padding='max_length', truncation=True, max_length=256))

In [19]:
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

In [20]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [21]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=bert_base,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

CODECARBON : No CPU tracking mode found. Falling back on CPU constant mode.


In [22]:
trainer.train()

/transformers/src/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.476800
1000,1.258500
1500,1.216100


In [94]:
# mlflow.set_tracking_uri('./mlruns')
# mlflow.set_experiment("Emotion_classfication")

In [45]:
# train_params = dict(
#     optimizer = 'Adam', # Adam, AdamW
#     learning_rate=5e-5,
#     weight_decay = None, # None, 1e-5 etc
#     patience = 5,
#     epochs = 30,
# )

In [ ]:
# from sklearn.metrics import accuracy_score

# mlflow.tensorflow.autolog()

# run_name = "baseline"

# with mlflow.start_run(run_name=run_name):
#     train_model(model,
#                 train_params,
#                 train_dataset,
#                 test_dataset)
    
#     y_pred = model.predict(test_dataset)
#     acc = accuracy_score(y_test, y_pred.argmax(axis=1))